# Security Exploratory Data Analysis

Statistical analysis of network telemetry to surface suspicious patterns.
No machine learning—focused on interpretable metrics that analysts can act on.

In [ ]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import matplotlib.pyplot as plt

from scripts.zeek_to_dataframe import load_zeek_log, CONN_SCHEMA, DNS_SCHEMA
from scripts.normalize import normalize_conn, normalize_dns, merge_normalized
from scripts.enrich_ti import ThreatIntel, enrich_ti

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
%matplotlib inline
plt.style.use("seaborn-v0_8-whitegrid")

## 1 — Load and Prepare Data

In [ ]:
ZEEK_DIR = Path("../data/zeek_logs/sample")

# Load and normalize
conn_raw = load_zeek_log(ZEEK_DIR / "conn.log", schema=CONN_SCHEMA)
dns_raw = load_zeek_log(ZEEK_DIR / "dns.log", schema=DNS_SCHEMA)

conn = normalize_conn(conn_raw)
dns = normalize_dns(dns_raw)

# Load TI and enrich
ti = ThreatIntel()
ti.load_ip_blacklist("../data/ti/sample_ips.txt")
ti.load_domain_blacklist("../data/ti/sample_domains.txt")

conn = enrich_ti(conn, ti, ip_column="dst_ip", domain_column=None)
dns = enrich_ti(dns, ti, ip_column="dst_ip", domain_column="dns_query")

# Add mock country data for demonstration
# In production, use: from scripts.enrich_geoip import enrich_geoip
country_map = {
    "93.184.216.34": "US",    # example.com
    "8.8.8.8": "US",          # Google DNS
    "8.8.4.4": "US",          # Google DNS
    "185.199.108.153": "US",  # GitHub
    "44.230.90.12": "US",     # AWS (suspicious)
    "44.230.90.13": "US",     # AWS (suspicious)
    "44.230.90.14": "US",     # AWS (suspicious)
    "151.101.1.140": "US",    # Reddit/Fastly
    "172.217.14.99": "US",    # Google
}
conn["dst_country"] = conn["dst_ip"].map(country_map).astype("string")
dns["dst_country"] = dns["dst_ip"].map(country_map).astype("string")

# Merge into unified timeline
unified = merge_normalized(conn, dns)

print(f"Connections: {len(conn)}")
print(f"DNS queries: {len(dns)}")
print(f"Unified events: {len(unified)}")

## 2 — Traffic Distribution by Protocol

In [ ]:
# Protocol breakdown
protocol_counts = conn["protocol"].value_counts()
protocol_bytes = conn.groupby("protocol")[["bytes_sent", "bytes_recv"]].sum()
protocol_bytes["total_bytes"] = protocol_bytes["bytes_sent"].fillna(0) + protocol_bytes["bytes_recv"].fillna(0)

print("=== Protocol Distribution (Connection Count) ===")
print(protocol_counts.to_string())
print()
print("=== Protocol Distribution (Bytes Transferred) ===")
print(protocol_bytes.to_string())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Connection count
protocol_counts.plot.pie(ax=axes[0], autopct="%1.0f%%", startangle=90)
axes[0].set_title("Connections by Protocol")
axes[0].set_ylabel("")

# Bytes transferred
protocol_bytes["total_bytes"].plot.pie(ax=axes[1], autopct="%1.0f%%", startangle=90)
axes[1].set_title("Bytes by Protocol")
axes[1].set_ylabel("")

plt.tight_layout()

In [ ]:
# Service breakdown within TCP
tcp_services = conn[conn["protocol"] == "tcp"]["service"].value_counts(dropna=False)
print("=== TCP Services ===")
print(tcp_services.to_string())
print()
print("Note: <NA> indicates connections where Zeek couldn't identify the application protocol")
print("      (often failed connections or encrypted traffic without SNI)")

## 3 — Top Destination Countries

In [ ]:
# Country distribution (external IPs only)
external_conn = conn[conn["dst_country"].notna()]

country_stats = external_conn.groupby("dst_country").agg(
    connections=("uid", "count"),
    unique_dst_ips=("dst_ip", "nunique"),
    bytes_sent=("bytes_sent", "sum"),
    bytes_recv=("bytes_recv", "sum"),
).sort_values("connections", ascending=False)

country_stats["bytes_sent"] = country_stats["bytes_sent"].fillna(0).astype(int)
country_stats["bytes_recv"] = country_stats["bytes_recv"].fillna(0).astype(int)

print("=== Traffic by Destination Country ===")
print(country_stats.to_string())

In [ ]:
# Visualization
if len(country_stats) > 0:
    fig, ax = plt.subplots(figsize=(10, 4))
    country_stats["connections"].head(10).plot.barh(ax=ax, color="steelblue")
    ax.set_xlabel("Connection Count")
    ax.set_title("Top 10 Destination Countries")
    ax.invert_yaxis()
    plt.tight_layout()

## 4 — Suspicious Pattern Detection (Statistics-Based)

Simple statistical methods to surface anomalies without machine learning.

### 4.1 — Connection State Analysis (Failed Connections)

In [ ]:
# Connection state distribution
# S0 = SYN sent, no response (potential scanning or blocked traffic)
# SF = Normal completion
# REJ = Connection rejected
# RSTO/RSTR = Reset by originator/responder

state_counts = conn["conn_state"].value_counts()
state_pct = (state_counts / len(conn) * 100).round(1)

state_analysis = pd.DataFrame({
    "count": state_counts,
    "percent": state_pct,
})

print("=== Connection State Distribution ===")
print(state_analysis.to_string())
print()

# Flag: High S0 rate indicates scanning or firewall blocks
s0_count = state_counts.get("S0", 0)
s0_rate = s0_count / len(conn) * 100
if s0_rate > 10:
    print(f"⚠️  ALERT: {s0_rate:.1f}% S0 (no response) — potential port scanning or blocked egress")
elif s0_rate > 5:
    print(f"⚡ WARNING: {s0_rate:.1f}% S0 — elevated failed connections")
else:
    print(f"✓ S0 rate ({s0_rate:.1f}%) within normal range")

In [ ]:
# Which source IPs have the most failed connections?
failed_states = ["S0", "REJ", "RSTO", "RSTR", "S1", "S2", "S3"]
failed_conn = conn[conn["conn_state"].isin(failed_states)]

if len(failed_conn) > 0:
    failed_by_src = failed_conn.groupby("src_ip").agg(
        failed_count=("uid", "count"),
        unique_dst_ips=("dst_ip", "nunique"),
        unique_dst_ports=("dst_port", "nunique"),
    ).sort_values("failed_count", ascending=False)
    
    print("=== Failed Connections by Source IP ===")
    print(failed_by_src.to_string())
    print()
    
    # Flag: Many unique dst_ports from one source = likely scanning
    scanners = failed_by_src[failed_by_src["unique_dst_ports"] > 3]
    if len(scanners) > 0:
        print("⚠️  Potential scanners (failed connections to many ports):")
        for ip in scanners.index:
            print(f"    {ip}: {scanners.loc[ip, 'unique_dst_ports']} unique ports")
else:
    print("✓ No failed connections detected")

### 4.2 — DNS Anomaly Detection (NXDOMAIN Analysis)

In [ ]:
# DNS response code distribution
rcode_counts = dns["dns_rcode"].value_counts()
rcode_pct = (rcode_counts / len(dns) * 100).round(1)

rcode_analysis = pd.DataFrame({
    "count": rcode_counts,
    "percent": rcode_pct,
})

print("=== DNS Response Code Distribution ===")
print(rcode_analysis.to_string())
print()

# Flag: High NXDOMAIN rate may indicate DGA malware
nxdomain_count = rcode_counts.get("NXDOMAIN", 0)
nxdomain_rate = nxdomain_count / len(dns) * 100
if nxdomain_rate > 30:
    print(f"⚠️  ALERT: {nxdomain_rate:.1f}% NXDOMAIN — potential DGA activity")
elif nxdomain_rate > 15:
    print(f"⚡ WARNING: {nxdomain_rate:.1f}% NXDOMAIN — elevated lookup failures")
else:
    print(f"✓ NXDOMAIN rate ({nxdomain_rate:.1f}%) within normal range")

In [ ]:
# Which hosts are generating NXDOMAIN responses?
nxdomain_queries = dns[dns["dns_rcode"] == "NXDOMAIN"]

if len(nxdomain_queries) > 0:
    nx_by_src = nxdomain_queries.groupby("src_ip").agg(
        nxdomain_count=("uid", "count"),
        unique_domains=("dns_query", "nunique"),
    ).sort_values("nxdomain_count", ascending=False)
    
    print("=== NXDOMAIN by Source IP ===")
    print(nx_by_src.to_string())
    print()
    
    # Show the actual failed domains
    print("=== Failed Domain Lookups ===")
    print(nxdomain_queries[["timestamp", "src_ip", "dns_query"]].to_string())
else:
    print("✓ No NXDOMAIN responses detected")

### 4.3 — Data Transfer Anomalies

In [ ]:
# Calculate bytes ratio: bytes_sent / bytes_recv
# High ratio (>> 1) = uploading data (potential exfiltration)
# Low ratio (<< 1) = downloading data (normal browsing)

conn_with_bytes = conn[
    (conn["bytes_sent"].notna()) & 
    (conn["bytes_recv"].notna()) &
    (conn["bytes_recv"] > 0)
].copy()

if len(conn_with_bytes) > 0:
    conn_with_bytes["bytes_ratio"] = conn_with_bytes["bytes_sent"] / conn_with_bytes["bytes_recv"]
    
    print("=== Data Transfer Statistics ===")
    print(f"Mean bytes_sent:  {conn_with_bytes['bytes_sent'].mean():,.0f}")
    print(f"Mean bytes_recv:  {conn_with_bytes['bytes_recv'].mean():,.0f}")
    print(f"Median bytes_ratio: {conn_with_bytes['bytes_ratio'].median():.2f}")
    print()
    
    # Flag connections with unusually high upload ratio
    high_upload = conn_with_bytes[conn_with_bytes["bytes_ratio"] > 1.0]
    if len(high_upload) > 0:
        print("=== High Upload Ratio Connections (potential exfiltration) ===")
        print(high_upload[["timestamp", "src_ip", "dst_ip", "dst_port", "bytes_sent", "bytes_recv", "bytes_ratio"]].to_string())
    else:
        print("✓ No high upload ratio connections detected")
else:
    print("Insufficient data for bytes analysis")

In [ ]:
# Top data senders (by bytes_sent to external IPs)
external_transfers = conn[conn["dst_country"].notna()].copy()

if len(external_transfers) > 0:
    top_senders = external_transfers.groupby("src_ip").agg(
        total_bytes_sent=("bytes_sent", "sum"),
        connection_count=("uid", "count"),
        unique_destinations=("dst_ip", "nunique"),
    ).sort_values("total_bytes_sent", ascending=False)
    
    top_senders["total_bytes_sent"] = top_senders["total_bytes_sent"].fillna(0).astype(int)
    
    print("=== Top Data Senders (to external IPs) ===")
    print(top_senders.to_string())

### 4.4 — Threat Intelligence Correlation

In [ ]:
# TI match summary
ti_matches = unified[unified["ti_match"] == True]

print("=== Threat Intelligence Matches ===")
print(f"Total events: {len(unified)}")
print(f"TI matches: {len(ti_matches)} ({len(ti_matches)/len(unified)*100:.1f}%)")
print()

if len(ti_matches) > 0:
    print("=== TI Matches by Log Type ===")
    print(ti_matches["log_type"].value_counts().to_string())
    print()
    
    print("=== Affected Source IPs ===")
    affected_hosts = ti_matches["src_ip"].value_counts()
    print(affected_hosts.to_string())
    print()
    
    print("=== TI Match Details ===")
    print(ti_matches[["timestamp", "log_type", "src_ip", "dst_ip", "dst_port", "dns_query"]].to_string())

### 4.5 — Beaconing Detection (Connection Timing)

In [ ]:
# Simple beaconing detection: look for repeated connections to same destination
# with regular intervals (C2 callback behavior)

conn_pairs = conn.groupby(["src_ip", "dst_ip", "dst_port"]).agg(
    connection_count=("uid", "count"),
    first_seen=("timestamp", "min"),
    last_seen=("timestamp", "max"),
).reset_index()

# Flag: Multiple connections to same destination = potential beaconing
repeated = conn_pairs[conn_pairs["connection_count"] > 1].sort_values("connection_count", ascending=False)

print("=== Repeated Connection Patterns ===")
if len(repeated) > 0:
    print(repeated.to_string())
    print()
    print("Note: Repeated connections to same dst_ip:dst_port may indicate:")
    print("  - Normal keepalive/polling (if known service)")
    print("  - C2 beaconing (if unknown/suspicious destination)")
else:
    print("No repeated connection patterns detected (sample too small)")

## 5 — Summary: Hosts Requiring Investigation

In [ ]:
# Aggregate all suspicious indicators per source IP
all_src_ips = unified["src_ip"].unique()

investigation_list = []

for src_ip in all_src_ips:
    host_data = unified[unified["src_ip"] == src_ip]
    host_conn = conn[conn["src_ip"] == src_ip]
    host_dns = dns[dns["src_ip"] == src_ip]
    
    flags = []
    
    # Check TI matches
    ti_count = host_data["ti_match"].sum()
    if ti_count > 0:
        flags.append(f"TI:{ti_count}")
    
    # Check failed connections
    failed = host_conn[host_conn["conn_state"].isin(["S0", "REJ"])]
    if len(failed) > 0:
        flags.append(f"FAIL:{len(failed)}")
    
    # Check NXDOMAIN
    nx = host_dns[host_dns["dns_rcode"] == "NXDOMAIN"]
    if len(nx) > 0:
        flags.append(f"NX:{len(nx)}")
    
    if flags:
        investigation_list.append({
            "src_ip": src_ip,
            "total_events": len(host_data),
            "flags": ", ".join(flags),
            "flag_count": len(flags),
        })

if investigation_list:
    invest_df = pd.DataFrame(investigation_list).sort_values("flag_count", ascending=False)
    print("=== HOSTS REQUIRING INVESTIGATION ===")
    print(invest_df.to_string(index=False))
    print()
    print("Flag legend:")
    print("  TI:N   = N threat intelligence matches")
    print("  FAIL:N = N failed connection attempts")
    print("  NX:N   = N NXDOMAIN responses (failed DNS)")
else:
    print("✓ No hosts flagged for investigation")